In [3]:
import re
import weaviate
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import WeaviateVectorStore
from langchain.agents import AgentOutputParser
from llama_index import get_response_synthesizer
from langchain.llms import OpenAI
from langchain.agents import AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.prompts import StringPromptTemplate
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from llama_index import ServiceContext
from llama_index.storage.storage_context import StorageContext
from llama_index import ServiceContext
from llama_index import QueryBundle
from llama_index.schema import NodeWithScore
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever
from typing import List
import openai
import streamlit as st

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/streamlit/elements/image.py:22: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [4]:
openai.api_key = st.secrets['OPENAI_API_KEY']

documents = SimpleDirectoryReader("data_MV").load_data()

# resource_owner_config = weaviate.AuthClientPassword(
#     username = st.secrets['WEAVIATE_UID'],
#     password = st.secrets['WEAVIATE_P'],
# )

# client = weaviate.Client("https://lawtech1-z4xd25ts.weaviate.network", auth_client_secret = resource_owner_config)
service_context = ServiceContext.from_defaults(llm = OpenAI(temperature = 0))
# vector_store = WeaviateVectorStore(weaviate_client = client)
# storage_context = StorageContext.from_defaults(vector_store = vector_store)
storage_context = StorageContext.from_defaults()
index = VectorStoreIndex.from_documents(documents, storage_context = storage_context)

template = """You are a Smart search agent. The Agent will ask you domain specific questions. The knowledge base provided to you have smart interpretibility if you specify keywords in your query to the tool [Example a query for two wheeler insurance rules should mention two wheelers].

        Action Input: the input to the action. Enhance the query such that it can  improve the performance of the model question answering model. Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps.to accurately complete the task. If the task is a question,the final step should almost always be 'Given the above steps taken,please respond to the users original question'.
        Then. self reflect on your answer, find faults and revise.
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times).
        Self Reflect: When one-third of the steps are responded, self reflect on the questions and the responses again and modify the results.
        Take more information from indices and self analyze the answer.
        Always self reflect on your answer, find faults and revise.
        Analyze {plex.intermediate_response} if it seems you would like to know more on the responses and if you would like to revisit any specific aspect
        or have any further questions, please let revise.
        Final Answer: the final answer to the original input question. Show the final answer or response  to the user with '$answer....$' in this manner. so as to rectify that it is the final answer.
        Pull up the related cases to the input's situation from the knowledge base. Display the filenames from {metadata} of the related cases and situations!!!
        Begin! Remember to be ethical and articulate when giving your final answer.

        Question: {input}
        {agent_scratchpad}"""

# prompt = CustomPromptTemplate(
#     template = template,
#     input_variables=["input", "intermediate_steps"]
# )

# vector_retriever = VectorIndexRetriever(index = index, prompt = prompt)
# llm = OpenAI(temperature = 0)

# custom_retriever = CustomRetriever(vector_retriever)
# response_synthesizer = get_response_synthesizer(service_context = service_context, response_mode = "tree_summarize")

query_engine = index.as_query_engine()

In [15]:
base_append = "\nList similar cases. If not exact, list similar cases. Give case title for each case."

In [16]:
query_engine.query('Drunk driving on highway night' + base_append)

Response(response='\n1. State of Punjab v. Balwinder Singh, AIR 2003 SC 3179 - Driving a vehicle under the influence of alcohol on a highway.\n2. State of Maharashtra v. Rajesh Kumar, AIR 2004 SC 486 - Driving a vehicle on a highway without a valid license.\n3. State of Maharashtra v. Rajesh Kumar, AIR 2004 SC 486 - Driving a vehicle on a highway without a valid insurance.\n4. State of Maharashtra v. Rajesh Kumar, AIR 2004 SC 486 - Driving a vehicle on a highway without a valid helmet.\n5. State of Maharashtra v. Rajesh Kumar, AIR 2004 SC 486 - Driving a vehicle on a highway without a valid pollution certificate.', source_nodes=[NodeWithScore(node=TextNode(id_='8f8a44a8-4976-40d5-87e2-d533bd569eba', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b539eb23-5aae-47bc-80d1-fc3758477b49', node_type=None, metadata={}, hash='7c13ce07c29170731df50a894db3dc09f7b953a35c403966ecfe

In [17]:
storage_context.persist(persist_dir = 'persist')